In [ ]:
# 🚗 Car Parts Image Generator (DreamShaper + CSV)
# Google Colab notebook

# --- 1. Setup ---
!pip install diffusers transformers accelerate safetensors pandas

import os
import pandas as pd
from diffusers import AutoPipelineForText2Image, DPMSolverMultistepScheduler
import torch
from google.colab import drive

# --- 2. Mount Google Drive ---
# This step requires authentication. Follow the prompts when you run the cell.
drive.mount('/content/drive')

# --- 3. Configuration ---
# Path to your CSV file on Google Drive
csv_path = "/content/drive/MyDrive/mods/default/car_parts/06_00_car_parts_fantasy_100.csv"
# Folder on Google Drive to save the generated images
output_dir = "/content/drive/MyDrive/mods/default/car_parts/06_00_car_parts_fantasy_100/generated_imgs"
os.makedirs(output_dir, exist_ok=True)

# --- 4. Load CSV ---
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} car parts.")

# --- 5. Load DreamShaper XL Lightning model ---
model_id = "lykon/dreamshaper-xl-lightning"
pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

# --- 6. Generate images ---
for idx, row in df.iterrows():
    part_id = row.get("Id")
    part_name = row.get("Part") or row.get("Name")
    part_description = row.get("Description")
    part_level = row.get("Level")
    
    prompt = (
        f"game asset car part: "
        f"{part_name}: {part_description}, mechanic and technical style, "
        f"futuristic, sci-fi and fantasy style, "
        f"intricate, highly detailed, sharp focus, vibrant colors, best quality, photorealistic, "
    )
    # -- The free model only handle prompt with 77 characters.
    
    negative_prompt = "worst quality, low quality, blurry, ugly, deformed, text, watermark, signature"

    try:
        image = pipe(
            prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=9,
            guidance_scale=3.0,
            width=240, # must be divisible by 8
            height=240
        ).images[0]
        filename = f"{part_id}_{part_name.replace(' ', '_')}.png"
        filepath = os.path.join(output_dir, filename)
        image.save(filepath)
        print(f"✅ Generated {part_name} → {filepath}")
    except Exception as e:
        print(f"❌ Failed for {part_name}: {e}")

print("🎉 Done! All images saved to Google Drive.")